# CS 109A/AC 209A/STAT 121A Data Science: Airbnb Project
## Step 1 - Analyze and Cleanse Data
**Harvard University**  
**Fall 2016**  
**Team**: Stephen Camera-Murray, Himani Garg, and Vijay Thangella  
**TF**: Christine Hwang  
  
**Due Date: ** Wednesday, December 14th, 2016 at 11:59pm

### Data Exploration

We begin by loading the datasets:  
- listings.csv.gz - the New York City Airbnb listing data from January 2015
- calendar.csv.gz - listing prices for specific dates-- to be analyzed for seasonality

Import libraries

In [1]:
# import libraries
import warnings
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt

# suppress warnings
warnings.filterwarnings ( 'ignore' )
%matplotlib inline

**Load and inspect the data**

In [2]:
# load listings data into a pandas df
listingsDF = pd.read_csv ( './datasets/listings.csv.gz' )

# display the first two rows
listingsDF.head ( n = 2 )

,id,scrape_id,last_scraped,name,picture_url,host_id,host_name,host_since,host_picture_url,street,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count
0,1069266,20150101184336,2015-01-02,Stay like a real New Yorker!,https://a0.muscache.com/pictures/50276484/larg...,5867023,Michael,2013-04-10,https://a2.muscache.com/ic/users/5867023/profi...,"East 53rd Street, New York, NY 10022, United S...",...,2013-04-28,2014-12-17,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1
1,1846722,20150101184336,2015-01-02,Apartment 20 Minutes Times Square,https://a1.muscache.com/pictures/35865039/larg...,2631556,Denise,2012-06-13,https://a2.muscache.com/ic/users/2631556/profi...,"West 155th Street, New York, NY, United States",...,2014-01-05,2014-12-29,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2


In [3]:
listingsDF.shape

(27392, 52)

The listings dataset is the main dataset we'll be using for prediction. It has 27,392 listings and 52 columns. As we can see from the names below, not all columns are suitable for prediction, but many of them are.

In [4]:
listingsDF.columns

Index([u'id', u'scrape_id', u'last_scraped', u'name', u'picture_url',
       u'host_id', u'host_name', u'host_since', u'host_picture_url', u'street',
       u'neighbourhood', u'neighbourhood_cleansed', u'city', u'state',
       u'zipcode', u'market', u'country', u'latitude', u'longitude',
       u'is_location_exact', u'property_type', u'room_type', u'accommodates',
       u'bathrooms', u'bedrooms', u'beds', u'bed_type', u'square_feet',
       u'price', u'weekly_price', u'monthly_price', u'guests_included',
       u'extra_people', u'minimum_nights', u'maximum_nights',
       u'calendar_updated', u'availability_30', u'availability_60',
       u'availability_90', u'availability_365', u'calendar_last_scraped',
       u'number_of_reviews', u'first_review', u'last_review',
       u'review_scores_rating', u'review_scores_accuracy',
       u'review_scores_cleanliness', u'review_scores_checkin',
       u'review_scores_communication', u'review_scores_location',
       u'review_scores_value', u'h

The calendar dataset has pricing data for listings at different times of the week. We'll look at this one to see if there are different prices for seasonality.  
  
**Note**: we added a buffer column to the end of the dataset since there's a bit of messiness in the data.

In [5]:
# load listings data into a pandas df
calendarDF = pd.read_csv ( './datasets/calendar.csv.gz', skiprows = 1, usecols = [ 0, 1, 2, 3, 4 ], names = [ 'listing_id', 'date', 'available', 'price', 'buffer' ] )

# display the first two rows
calendarDF.head ( n = 2 )

,listing_id,date,available,price,buffer
0,3604481,2015-01-01,t,$600.00,NaN
1,3604481,2015-01-02,t,$600.00,NaN


In [6]:
calendarDF.shape

(9998080, 5)

#### Data Cleansing

**Listing dataset**

We notice that some listings have weekly and monthly rates, but almost all listings have a daily rate. We start by converting the price column to a float so we can work with it as a number. Next, we remove all columns that we don't plan to use for prediction: 'scrape_id', 'last_scraped', 'picture_url', 'host_id', 'host_name', 'host_picture_url', 'street', 'neighbourhood', 'city', 'state', 'zipcode', 'market', 'country', 'latitude', 'longitude', 'is_location_exact', 'weekly_price', 'monthly_price', 'extra_people', 'calendar_updated', 'calendar_last_scraped'.

In [7]:
# convert the price column to a float
listingsDF [ 'price' ] = listingsDF [ 'price' ].replace ( '[\$,)]', '', regex = True ).replace ( '[(]', '-', regex = True ).astype ( float )

# drop columns we don't need
listingsDF.drop ( [ 'scrape_id', 'last_scraped', 'picture_url', 'host_id', 'host_name', 'host_picture_url', 'street', 'neighbourhood', 'city', 'state', 'zipcode', 'market', 'country', 'latitude', 'longitude', 'is_location_exact', 'weekly_price', 'monthly_price', 'extra_people', 'calendar_updated', 'calendar_last_scraped' ], axis = 1, inplace = True )

# rename the neighbourhood_cleansed column to neighborhood
listingsDF = listingsDF.rename ( columns = { 'neighbourhood_cleansed' : 'neighborhood' } )

# display the first two rows
listingsDF.head ( n = 2 )

,id,name,host_since,neighborhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,...,first_review,last_review,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count
0,1069266,Stay like a real New Yorker!,2013-04-10,Midtown East,Apartment,Entire home/apt,2,1.0,1.0,1.0,...,2013-04-28,2014-12-17,86.0,9.0,7.0,9.0,9.0,10.0,9.0,1
1,1846722,Apartment 20 Minutes Times Square,2012-06-13,Hamilton Heights,Apartment,Entire home/apt,10,1.0,3.0,3.0,...,2014-01-05,2014-12-29,85.0,8.0,8.0,9.0,8.0,7.0,8.0,2


Next, we'll convert all of the date fields to an integer representing the month age so we can better work with them.  
  
**Note**: the first_review and last_review columns may be blank. If they are we'll penalize them by setting it to the "worst" value in the data based on our assumption that the worst case for a first review is it's recent, or the lastest date in our data, and the last review is old, or the earliest date in our data.

In [8]:
# fill in missing dates with the "worst" value
listingsDF [ "first_review" ][ listingsDF [ "first_review" ].isnull() ] = listingsDF [ "first_review" ][ listingsDF [ "first_review" ].notnull() ].max()
listingsDF [ "last_review"  ][ listingsDF [ "last_review"  ].isnull() ] = listingsDF [ "last_review"  ][ listingsDF [ "last_review"  ].notnull() ].min()

# create new date fields based on a months passed
listingsDF [ "months_as_host" ] = ( ( 2014 - listingsDF [ "host_since" ].str [ : 4 ].astype ( int ) ) * 12 ) + ( 13 - listingsDF [ "host_since" ].str [ 5 : 7 ].astype ( int ) )
listingsDF [ "months_since_first_review" ] = ( ( 2014 - listingsDF [ "first_review" ].str [ : 4 ].astype ( int ) ) * 12 ) + ( 13 - listingsDF [ "first_review" ].str [ 5 : 7 ].astype ( int ) )
listingsDF [ "months_since_last_review"  ] = ( ( 2014 - listingsDF [ "last_review"  ].str [ : 4 ].astype ( int ) ) * 12 ) + ( 13 - listingsDF [ "last_review"  ].str [ 5 : 7 ].astype ( int ) )

# drop columns we don't need
listingsDF.drop ( [ 'host_since', 'first_review', 'last_review' ], axis = 1, inplace = True )

# display the first two rows
listingsDF.head ( n = 2 )

,id,name,neighborhood,property_type,room_type,accommodates,bathrooms,bedrooms,beds,bed_type,...,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,host_listing_count,months_as_host,months_since_first_review,months_since_last_review
0,1069266,Stay like a real New Yorker!,Midtown East,Apartment,Entire home/apt,2,1.0,1.0,1.0,Real Bed,...,9.0,7.0,9.0,9.0,10.0,9.0,1,21,21,1
1,1846722,Apartment 20 Minutes Times Square,Hamilton Heights,Apartment,Entire home/apt,10,1.0,3.0,3.0,Real Bed,...,8.0,8.0,9.0,8.0,7.0,8.0,2,31,12,1


**Calendar listing data**

We start by converting the price to a float so we can work with it as a number and drop the buffer column.

In [60]:
# convert the price column to a float
calendarDF [ 'price' ] = calendarDF [ 'price' ].replace ( '[\$,)]', '', regex = True ).replace ( '[(]', '-', regex = True ).astype ( float )

# drop the buffer column
calendarDF.drop ( [ 'buffer' ], axis = 1, inplace = True )

# display the first two rows
calendarDF.head ( n = 2 )

,listing_id,date,available,price
0,3604481,2015-01-01,t,600.0
1,3604481,2015-01-02,t,600.0


Next, we add a seasonal column by extracting the month and applying some simple logic.

In [61]:
# create a season column for the calendar listing prices by: 1) extract the month to a separate column,
# and 2) create a season column based on the month
calendarDF [ "month" ] = calendarDF [ "date" ].str [ 5 : 7 ].astype ( int )
calendarDF [ "season" ] = "Winter"
calendarDF [ "season" ][ ( calendarDF [ 'month' ] >= 3 ) &  ( calendarDF [ 'month' ] <= 5 ) ] = "Spring"
calendarDF [ "season" ][ ( calendarDF [ 'month' ] >= 6 ) &  ( calendarDF [ 'month' ] <= 8 ) ] = "Summer"
calendarDF [ "season" ][ ( calendarDF [ 'month' ] >= 9 ) &  ( calendarDF [ 'month' ] <= 11 ) ] = "Fall"

# display the first two rows
calendarDF.head ( n = 2 )

,listing_id,date,available,price,month,season
0,3604481,2015-01-01,t,600.0,1,Winter
1,3604481,2015-01-02,t,600.0,1,Winter


Finally, we group the listings and seasons to get the mean listing price for the season.

In [62]:
# get rid of rows with blank or meaningless prices
calendarDF = calendarDF [ np.isfinite ( calendarDF [ 'price' ] ) ]

# create a seasonal dataframe with average seasonal pricing for each listing
seasonalDF = calendarDF.groupby ( [ 'listing_id', 'season' ] ) [ 'price' ].mean().to_frame ( name = 'price' ).reset_index()

seasonalDF.head ( n = 2 )

,listing_id,season,price
0,105,Fall,363.285714
1,105,Spring,360.956522
